<a href="https://colab.research.google.com/github/DanVijl/NICU_fluid-nutrient/blob/main/feeding_agent_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🍼 Feeding Agent voor de NICU

Dit notebook rekent automatisch enterale en parenterale voeding voor prematuren, op basis van GA, DOL, gewicht en voedingstype.

Je kunt zelf toevoegen: BMF, Fantomalt, Protein Fortifier en Solagen.

In [29]:
# 📦 Installatie niet nodig op Colab
# (maar hieronder placeholder voor lokale installatie)
# !pip install pandas

In [30]:
# 📚 Functies laden
import pandas as pd

FEEDS = {
    "breast": {"energy": 0.67, "protein": 0.012, "fat": 0.034, "carb": 0.065},
    "donor": {"energy": 0.55, "protein": 0.009, "fat": 0.028, "carb": 0.065},
    "nenatal_start": {"energy": 0.80, "protein": 0.026, "fat": 0.039, "carb": 0.084},
    "nutrilon_standard": {"energy": 0.66, "protein": 0.013, "fat": 0.034, "carb": 0.073},
}

FORTIFIERS_PER_G = {
    "bmf": {"energy": 4.25, "protein": 0.325, "fat": 0.175, "carb": 0.375},
    "protein_fort": {"energy": 10.0, "protein": 2.2, "fat": 0.0, "carb": 0.0},
    "fantomalt": {"energy": 4.0, "protein": 0.0, "fat": 0.0, "carb": 1.0},
    "solagen": {"energy": 9.0, "protein": 0.0, "fat": 1.0, "carb": 0.0},
}

PARENTERAL = {
    "NICUmix": {"energy": 0.29, "protein": 0.0743, "fat": 0.0, "carb": 0.0},
    "SMOF20":  {"energy": 1.80, "protein": 0.0, "fat": 0.20, "carb": 0.0},
    "G10":     {"energy": 0.40, "protein": 0.0, "fat": 0.0, "carb": 0.10},
}

def targets(ga_weeks: float, dol: int):
    if ga_weeks < 30:
        return {"fluid_min": 140 if dol>=5 else 120,
                "fluid_max": 160 if dol>=5 else 140,
                "protein": 3.5 if dol>=5 else 3.0,
                "fat": 3.5 if dol>=5 else 3.0,
                "carb": 9.0 if dol>=5 else 8.0}
    else:
        return {"fluid_min": 130, "fluid_max": 150, "protein": 3.0,
                "fat": 3.0, "carb": 8.0}

def enriched_feed(feed_type, vol_mlkg, bmf_pct=None,
                  protein_fort_pct=None, fantomalt_pct=None,
                  solagen_pct=None):
    comp = FEEDS[feed_type].copy()
    if bmf_pct is None and vol_mlkg >= 100:
        bmf_pct = 4 if feed_type == "breast" else 5
    fortifiers = {
        "bmf": bmf_pct,
        "protein_fort": protein_fort_pct,
        "fantomalt": fantomalt_pct,
        "solagen": solagen_pct,
    }
    for fort, pct in fortifiers.items():
        if pct is None or pct == 0:
            continue
        grams = pct / 100
        for k in comp:
            comp[k] += FORTIFIERS_PER_G[fort][k] * grams
    return {k: comp[k]*vol_mlkg for k in comp}, fortifiers

def compute_parenteral(ga_weeks, dol, weight_kg,
                       feed_vol_ml, feeds_per_day,
                       feed_type="breast",
                       bmf_pct=None, protein_fort_pct=None,
                       fantomalt_pct=None, solagen_pct=None):
    t = targets(ga_weeks, dol)
    enteral_mlkg = feeds_per_day * feed_vol_ml / weight_kg
    enteral_macros, fortifiers = enriched_feed(
        feed_type, enteral_mlkg, bmf_pct, protein_fort_pct,
        fantomalt_pct, solagen_pct)
    protein_gap = max(t["protein"] - enteral_macros["protein"], 0)
    fat_gap = max(t["fat"] - enteral_macros["fat"], 0)
    vol_nicu = protein_gap / PARENTERAL["NICUmix"]["protein"] if protein_gap>0 else 0
    vol_smof = fat_gap / PARENTERAL["SMOF20"]["fat"] if fat_gap>0 else 0
    used_fluid = enteral_mlkg + vol_nicu + vol_smof
    vol_g10 = max(t["fluid_max"] - used_fluid, 0)
    volumes = {"NICUmix":vol_nicu,"SMOF20":vol_smof,"G10":vol_g10}
    totals = enteral_macros.copy()
    for prod, vol in volumes.items():
        for k in totals:
            totals[k]+=PARENTERAL[prod][k]*vol
    glucose_rate = round(totals["carb"]*1000/1440, 2)
    summary = pd.DataFrame({
        "Parameter":["Fluid (mL/kg)","Energy (kcal)","Protein (g)",
                     "Fat (g)","Carbs (g)","Glucose (mg/kg/min)"],
        "Total/day":[round(enteral_mlkg + vol_nicu + vol_smof + vol_g10,2),
                     round(totals["energy"],2), round(totals["protein"],2),
                     round(totals["fat"],2), round(totals["carb"],2), glucose_rate],
        "Target":[f'{t["fluid_min"]}-{t["fluid_max"]}',
                  "-", t["protein"], t["fat"], t["carb"], "7–9"]
    })
    return pd.DataFrame(volumes, index=["mL/kg/d"]).T.round(1), summary, fortifiers

In [31]:
# ▶️ Vul je patiëntgegevens in:
volumes, macros, fortifiers = compute_parenteral(
    ga_weeks=27,
    dol=25,
    weight_kg=0.99,
    feed_vol_ml=13,
    feeds_per_day=12,
    feed_type="breast",  # of: donor, nenatal_start, nutrilon_standard
    bmf_pct=None,         # laat op None voor automatische toevoeging vanaf 100 mL/kg
    protein_fort_pct=None,
    fantomalt_pct=None,
    solagen_pct=None
)

print("📦 Parenterale volumes")
display(volumes)
print("🔬 Macro-overzicht")
display(macros)
print("🧪 Fortifiers toegepast")
display(pd.DataFrame.from_dict(fortifiers, orient="index", columns=["g per 100 mL"]))

📦 Parenterale volumes


,mL/kg/d
NICUmix,0.0
SMOF20,0.0
G10,2.4


🔬 Macro-overzicht


,Parameter,Total/day,Target
0,Fluid (mL/kg),160.00,140-160
1,Energy (kcal),133.33,-
2,Protein (g),3.94,3.5
3,Fat (g),6.46,3.5
4,Carbs (g),12.85,9.0
5,Glucose (mg/kg/min),8.92,7–9


🧪 Fortifiers toegepast


,g per 100 mL
bmf,4.0
protein_fort,NaN
fantomalt,NaN
solagen,NaN
